# Homework 6

#### Add packages

In [ ]:
import altair as alt
import pandas as pd

#### Load the dataset

In [ ]:
file_path = 'https://anxuanzi.github.io/assets/hw6/licenses_fall2022.csv'
df = pd.read_csv(file_path)

#### Display the first few rows of the dataset


In [ ]:
df.head()

#### Selecting a subset of the columns for visualization


In [ ]:
columns_of_interest = [
    'License Type', 'Description', 'License Status', 'Business', 'Title', 'First Name',
    'Last Name', 'License Number', 'Discipline Reason', 'Discipline Start Date',
    'Discipline End Date'
]
df_subset = df[columns_of_interest]

#### Data Cleaning


Convert Discipline Start/End Dates to datetime

Filtering rows where Discipline Start Date and Discipline End Date are not 'None'

In [ ]:
disciplinary_df = df_subset[
    (df_subset['Discipline Start Date'] != 'None') &
    (df_subset['Discipline End Date'] != 'None')
    ]

# Converting Discipline Start Date and End Date to datetime
disciplinary_df['Discipline Start Date'] = pd.to_datetime(disciplinary_df['Discipline Start Date'], errors='coerce')
disciplinary_df['Discipline End Date'] = pd.to_datetime(disciplinary_df['Discipline End Date'], errors='coerce')

# Dropping rows where dates could not be parsed
disciplinary_df_cleaned = disciplinary_df.dropna(subset=['Discipline Start Date', 'Discipline End Date'])

# Creating the First Plot: License Types by Status
license_status_counts = df_subset.groupby(['License Type', 'License Status']).size().reset_index(name='Counts')



#### Altair Bar Chart for License Types by Status

In [ ]:
bar_chart = alt.Chart(license_status_counts).mark_bar().encode(
    x=alt.X('License Type:N', sort='-y', title='License Type'),
    y=alt.Y('Counts:Q', title='Number of Licenses'),
    color=alt.Color('License Status:N', title='License Status'),
    tooltip=['License Type', 'License Status', 'Counts']
).properties(
    width='container',
    height=400,
    title='Number of Licenses by Type and Status'
).interactive()

# adding more interactivity

license_type_dropdown = alt.binding_select(options=list(license_status_counts['License Type'].unique()), name='Select License Type: ')
type_selection = alt.selection_point(fields=['License Type'], bind=license_type_dropdown, name='license_selection')

license_status_dropdown = alt.binding_select(
    options=list(license_status_counts['License Status'].unique()),
    name='Select License Status: '
)
status_selection = alt.selection_point(fields=['License Status'], bind=license_status_dropdown)

interactive_bar_chart = bar_chart.add_params(type_selection, status_selection).encode(
    color=alt.condition(
        type_selection & status_selection,
        'License Status:N',
        alt.value('lightgray')
    ),
    opacity=alt.condition(
        type_selection & status_selection,
        alt.value(1),
        alt.value(0.2)
    )
)

#### Timeline of Licenses with Disciplinary Actions

In [ ]:
disciplinary_chart = alt.Chart(disciplinary_df_cleaned).mark_line().encode(
    x=alt.X('yearmonth(Discipline Start Date):T', title='Discipline Start Date'),
    x2='yearmonth(Discipline End Date):T',
    y=alt.Y('License Type:N', title='License Type'),
    color=alt.Color('License Type:N', title='License Type'),
    tooltip=['License Type', 'Discipline Reason', 'Discipline Start Date', 'Discipline End Date']
).properties(
    width='container',
    height=400,
    title='Timeline of Licenses with Disciplinary Actions'
).interactive()

#### Display the plots

In [ ]:
interactive_bar_chart.save('../assets/hw6/number_of_licenses_by_type_and_status.json')
interactive_bar_chart

In [ ]:
disciplinary_chart.save('../assets/hw6/timeline_of_licenses_with_disciplinary_actions.json')
disciplinary_chart